In [1]:
import json
import os
import re
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [2]:
stations_dir = 'data/info'
status_dir = 'data/status'
free_bike_dir = 'data/free_bike'

In [3]:
stations_files = [file_name for file_name in os.listdir(stations_dir)]
status_files = [file_name for file_name in os.listdir(status_dir)]
free_bike_files = [file_name for file_name in os.listdir(free_bike_dir)]

In [ ]:
stations_data_list = []

for file in tqdm(stations_files, desc='file'):
    
    with open(f'data/info/{file}', 'r') as file_object:
        try:
            stations_json_load = json.load(file_object)
        except json.JSONDecodeError as e:
            print(f'error on {file}: {e}')
            continue

    time_stations_data = (
        pd.json_normalize(
            data=stations_json_load,
            record_path= [
                ['data','stations']
            ],
            meta='last_updated',
        )
        .filter(items=[
            'last_updated',
            'station_id',
            'short_name',
            'name',
            'capacity',
            'lat',
            'lon'
        ])
    )

    file_timestamp = int(re.search('\d*',file).group())

    time_stations_data['status_last_updated_fetched_timestamp'] = file_timestamp 


    stations_data_list.append(time_stations_data)

stations_data = pd.concat(stations_data_list)

In [ ]:
status_data_list = []

for file in tqdm(status_files):

    with open(f'data/status/{file}', 'r') as file_object:
        try:
            status_json_load = json.load(file_object)
        except json.JSONDecodeError as e:
            print(f'error on {file}: {e}')
            continue

    time_status_data = pd.json_normalize(
        data=status_json_load,
        record_path= [
            ['data','stations']
        ],
        meta='last_updated',
    ).filter(items=[
        'last_updated',
        'station_id',
        'station_status',
        'is_renting',
        'is_returning',
        'num_docks_available',
        'num_bikes_available',
        'num_ebikes_available',
        'num_bikes_disabled',
        'num_docks_disabled',
        'num_ebikes_disabled',
        'valet.active'
    ])

    status_data_list.append(time_status_data)


status_data = pd.concat(status_data_list)


localize times

In [6]:
status_data['last_updated'] = (
    status_data['last_updated']
    .apply(pd.Timestamp, unit='s', tz='America/New_York')
)

stations_data['status_last_updated_fetched_timestamp'] = (
    stations_data['status_last_updated_fetched_timestamp']
    .apply(pd.Timestamp, unit='s', tz='America/New_York')
)

check that stations data timestamps have matching status data timestamps

In [7]:
stations_data['station_id'] = stations_data['station_id'].astype(str)
status_data['station_id'] = status_data['station_id'].astype(str)

In [8]:
assert stations_data['status_last_updated_fetched_timestamp'].isin(status_data['last_updated']).all()

In [9]:
dataset = (
    stations_data
    .merge(
        status_data, 
        left_on=['status_last_updated_fetched_timestamp','station_id'],
        right_on=['last_updated','station_id'],
        how='inner',
        suffixes=['_stations',None]
    )
)

check that each station_id is a unique physical location

In [10]:
assert dataset.groupby('station_id')['lat'].nunique().max() == 1

In [11]:
stations_locations = (
    dataset
    .drop_duplicates(subset='station_id')
    .set_index('station_id')
    [['lat','lon']]
)

In [13]:
dataset = (
    dataset
    .set_index(['last_updated','station_id'])
    .drop(columns=[
        'name',
        'lat','lon',
        'status_last_updated_fetched_timestamp',
        'last_updated_stations',
        ])
)

In [ ]:
dataset.head(10)

save out

In [16]:
# Save dataset as CSV
dataset.to_csv('station_status.csv', index=True)


In [ ]:

# # Load your local dataset (replace 'station_status.csv' with the path to your dataset)
df = pd.read_csv("station_status.csv")
df.dtypes